In [ ]:
# This does basic reductions (bias, dark, flat correction, etc.)

# Created 2021 Oct 19 by E.S.

In [1]:
from astropy.io import fits
import numpy as np
import glob
import os
import matplotlib.pyplot as plt

In [14]:
# integration time to scale frames to (usually this is the integration time of the light images)
N = 4. # (seconds; take care to make sure the darks were taken at this integration time, since they are
        # not rescaled here)

# directories of files to use

parent_stem = "/Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019"
flat_stem = parent_stem + "/flats_for_master_flat"
light_stem = parent_stem + "/sci_data_to_reduce"
processed_stem = parent_stem + "/calibrated_sci_data_preregistration"

master_bias_file_name = parent_stem + "/master_bias_20211019.fits"

# this N-second dark has bias already subtracted
master_dark_file_name = parent_stem + "/master_dark_bias_subtd_20211019.fits"

In [3]:
# read in master dark (already bias-subtracted)
master_dark_obj = fits.open(master_dark_file_name)
master_dark = master_dark_obj[0].data

# read in master bias
master_bias_obj = fits.open(master_bias_file_name)
master_bias = master_bias_obj[0].data

In [4]:
file_list_lights_src = glob.glob(light_stem + "/*fit")
file_list_flats_src = glob.glob(flat_stem + "/*fit")

In [9]:
## make a master flat

# initialize cube
cube_flats = np.nan*np.ones((512,512,len(file_list_flats_src)))

# loop over the flats, calibrate and rescale as needed, and assemble into cube
for i in range(0,len(file_list_flats_src)):
    
    this_raw_flat = fits.open(file_list_flats_src[i])

    # subtract bias from all the raw flat frames
    flat_bias_subt = np.subtract(this_raw_flat[0].data,master_bias)

    # rescale flats to be equivalent to N-sec integration time (integration time of science frames and dark)
    # n.b. flat integration time is under keyword 'EXPTIME'
    exp_time = this_raw_flat[0].header["EXPTIME"]
    flat_bias_subt_rescaled = np.multiply((N/exp_time),flat_bias_subt)

    # subtract the N-sec master dark (which is already bias-subtracted) from the N-sec-equivalent flats
    flat_bias_subt_rescaled_dark_subted = np.subtract(flat_bias_subt_rescaled,master_dark)

    # put dark-subtracted flats into a cube
    cube_flats[:,:,i] = flat_bias_subt_rescaled_dark_subted

In [10]:
# take 2D median across flats, then divide by median value across that 2D array to get one master flat
median_unnormzed_flat = np.median(cube_flats, axis=2)
master_flat = np.divide(median_unnormzed_flat,np.median(median_unnormzed_flat))

In [11]:
# write out, in case

hdu = fits.PrimaryHDU(master_flat)
hdu.writeto('junk_master_flat.fits')

In [15]:
# read in lights, subtract bias, subtract dark, and divide by master flat
for j in range(0,len(file_list_lights_src)):
    
    # read in raw light frames (which have no bias or dark subtraction)
    light_obj_prebias = fits.open(file_list_lights_src[j])
    light_prebias = light_obj_prebias[0].data
    
    # subtract bias
    light_predark = np.subtract(light_prebias,master_bias)
    
    # subtract dark (which itself is bias-subtracted)
    light = np.subtract(light_predark,master_dark)
    
    # flat-fielding
    light_ffed = np.divide(light,master_flat)
    
    # write out, while keeping same header
    write_file_name = processed_stem + "/" + os.path.basename(file_list_lights_src[j])
    hdu2 = fits.PrimaryHDU(light_ffed, header=light_obj_prebias[0].header)
    hdu2.writeto(write_file_name, overwrite=True)
    
    print("Wrote " + write_file_name)

Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-0908.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-0934.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-1394.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-1380.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-0920.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-0713.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-0075.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrate

Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-0869.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-1034.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-1020.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-0458.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-0470.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-1008.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-0316.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrate

Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-0857.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-1585.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-0843.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-0670.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-1208.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-0116.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-0102.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrate

Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-1345.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-1423.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-1437.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-0729.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-1351.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-0067.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-1379.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrate

Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-1146.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-1191.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-1185.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-0714.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-0072.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-1378.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-0066.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrate

...

Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-1439.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-0733.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-0055.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-0900.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-0914.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-0082.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-0928.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrate

Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-0877.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-0687.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-0693.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-1002.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-1016.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-0308.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-0446.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrate

Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-0442.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-0873.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-0867.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-0697.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-0683.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-1210.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-0668.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrate

Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-1373.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-1415.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-1401.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-0079.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-1367.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-0051.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-1429.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrate

Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-0468.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-1004.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-0497.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-0483.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-0246.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-1158.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrated_sci_data_preregistration/toi_3760_4s-0520.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211019/calibrate